[View in Colaboratory](https://colab.research.google.com/github/thundercat95/AutoML/blob/master/SMAC.ipynb)

# SMAC on Python3

### Installation and Uploads

In [0]:
!pip install openml
!apt-get install build-essential swig
!pip install smac

In [0]:
from google.colab import files

uploaded = files.upload()

##files.upload returns a dictionary of the files which were uploaded. The 
##dictionary is keyed by the file name, the value is the data which was 
##uploaded.

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))

Saving creditopenml.csv to creditopenml.csv
User uploaded file "creditopenml.csv" with length 150828752 bytes


### Imports and datasets

In [0]:
import numpy as np
import pandas as pd
import time
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn import decomposition
import openml as oml
apikey = '2f6c58f4f6d62ea93dd32764ec88089c'
oml.config.apikey = apikey

# Import ConfigSpace and different types of parameters
from smac.configspace import ConfigurationSpace
from ConfigSpace.hyperparameters import CategoricalHyperparameter, UniformFloatHyperparameter, UniformIntegerHyperparameter
from ConfigSpace.conditions import InCondition

# Import SMAC-utilities
from smac.tae.execute_func import ExecuteTAFuncDict
from smac.scenario.scenario import Scenario
from smac.facade.smac_facade import SMAC

In [0]:
# We load the iris-dataset
iris = datasets.load_iris()

# We load Credit Card dataset from openml
creditcard = oml.datasets.get_dataset(1597)

### SVM with SMAC on iris data

In [0]:
# Build Configuration Space which defines all parameters and their ranges
cs = ConfigurationSpace()

In [4]:
# We define a few possible types of SVM-kernels and add them as "kernel" to our cs
kernel = CategoricalHyperparameter("kernel", ["linear", "rbf", "poly", "sigmoid"], default_value="poly")
cs.add_hyperparameter(kernel)

kernel, Type: Categorical, Choices: {linear, rbf, poly, sigmoid}, Default: poly

In [5]:
# There are some hyperparameters shared by all kernels
C = UniformFloatHyperparameter("C", 0.001, 1000.0, default_value=1.0)
shrinking = CategoricalHyperparameter("shrinking", ["true", "false"], default_value="true")
cs.add_hyperparameters([C, shrinking])

[C, Type: UniformFloat, Range: [0.001, 1000.0], Default: 1.0,
 shrinking, Type: Categorical, Choices: {true, false}, Default: true]

In [6]:
# Others are kernel-specific, so we can add conditions to limit the searchspace
degree = UniformIntegerHyperparameter("degree", 1, 5, default_value=3)     # Only used by kernel poly
coef0 = UniformFloatHyperparameter("coef0", 0.0, 10.0, default_value=0.0)  # poly, sigmoid
cs.add_hyperparameters([degree, coef0])
use_degree = InCondition(child=degree, parent=kernel, values=["poly"])
use_coef0 = InCondition(child=coef0, parent=kernel, values=["poly", "sigmoid"])
cs.add_conditions([use_degree, use_coef0])

[degree | kernel in {'poly'}, coef0 | kernel in {'poly', 'sigmoid'}]

In [7]:
# This also works for parameters that are a mix of categorical and values from a range of numbers
# For example, gamma can be either "auto" or a fixed float
gamma = CategoricalHyperparameter("gamma", ["auto", "value"], default_value="auto")  # only rbf, poly, sigmoid
gamma_value = UniformFloatHyperparameter("gamma_value", 0.0001, 8, default_value=1)
cs.add_hyperparameters([gamma, gamma_value])

[gamma, Type: Categorical, Choices: {auto, value}, Default: auto,
 gamma_value, Type: UniformFloat, Range: [0.0001, 8.0], Default: 1.0]

In [8]:
# We only activate gamma_value if gamma is set to "value"
cs.add_condition(InCondition(child=gamma_value, parent=gamma, values=["value"]))
# And again we can restrict the use of gamma in general to the choice of the kernel
cs.add_condition(InCondition(child=gamma, parent=kernel, values=["rbf", "poly", "sigmoid"]))

gamma | kernel in {'rbf', 'poly', 'sigmoid'}

In [0]:
def svm_from_cfg(cfg):
    """ Creates a SVM based on a configuration and evaluates it on the
    iris-dataset using cross-validation.

    Parameters:
    -----------
    cfg: Configuration (ConfigSpace.ConfigurationSpace.Configuration)
        Configuration containing the parameters.
        Configurations are indexable!

    Returns:
    --------
    A crossvalidated mean score for the svm on the loaded data-set.
    """
    # For deactivated parameters, the configuration stores None-values.
    # This is not accepted by the SVM, so we remove them.
    cfg = {k : cfg[k] for k in cfg if cfg[k]}
    # We translate boolean values:
    cfg["shrinking"] = True if cfg["shrinking"] == "true" else False
    # And for gamma, we set it to a fixed value or to "auto" (if used)
    if "gamma" in cfg:
        cfg["gamma"] = cfg["gamma_value"] if cfg["gamma"] == "value" else "auto"
        cfg.pop("gamma_value", None)  # Remove "gamma_value"

    clf = svm.SVC(**cfg, random_state=42)
    scores = cross_val_score(clf, iris.data, iris.target, cv=5)
    return 1-np.mean(scores)  # Minimize!

In [0]:
# Scenario object
scenario = Scenario({"run_obj": "quality",   # we optimize quality (alternatively runtime)
                     "runcount-limit": 200,  # maximum function evaluations
                     "cs": cs,               # configuration space
                     "deterministic": "true"
                     })

In [23]:
# Optimize, using a SMAC-object
print("Optimizing! Depending on your machine, this might take a few minutes.")
smac = SMAC(scenario=scenario, rng=np.random.RandomState(42),
        tae_runner=svm_from_cfg)
incumbent = smac.optimize()
inc_value = svm_from_cfg(incumbent)
print("Optimized Value: %.2f" % (inc_value))
print("Best parameter : ", incumbent.get_dictionary())


Optimizing! Depending on your machine, this might take a few minutes.
Optimized Value: 0.01
Best parameter :  {'C': 1.3375368604838664, 'kernel': 'linear', 'shrinking': 'false'}


### SVM with SMAC on Credit Card dataset

In [0]:
# Scenario object
scenario = Scenario({"run_obj": "quality",   # we optimize quality (alternatively runtime)
                     "runcount-limit": 5,    # maximum function evaluations
                     "cs": cs,               # configuration space
                     "deterministic": "true"
                     })

In [0]:
def svm_from_cfg(cfg):
    """ Creates a SVM based on a configuration and evaluates it on the
    iris-dataset using cross-validation.

    Parameters:
    -----------
    cfg: Configuration (ConfigSpace.ConfigurationSpace.Configuration)
        Configuration containing the parameters.
        Configurations are indexable!

    Returns:
    --------
    A crossvalidated mean score for the svm on the loaded data-set.
    """
    # For deactivated parameters, the configuration stores None-values.
    # This is not accepted by the SVM, so we remove them.
    cfg = {k : cfg[k] for k in cfg if cfg[k]}
    # We translate boolean values:
    cfg["shrinking"] = True if cfg["shrinking"] == "true" else False
    # And for gamma, we set it to a fixed value or to "auto" (if used)
    if "gamma" in cfg:
        cfg["gamma"] = cfg["gamma_value"] if cfg["gamma"] == "value" else "auto"
        cfg.pop("gamma_value", None)  # Remove "gamma_value"
    
    X, y = creditcard.get_data(target=creditcard.default_target_attribute)
    y[y == "'0'"]=0
    y[y == "'1'"]=1
    
    print(cfg)
    ICA = decomposition.FastICA(tol=0.2)
    X_new = ICA.fit_transform(X)
    clf = svm.SVC(**cfg, random_state=42)
    scores = cross_val_score(clf, X_new, y, cv=5, scoring="roc_auc")
    print(np.mean(scores))
    return 1-np.mean(scores)  # Minimize!

In [13]:
# Optimize, using a SMAC-object
print("Optimizing! Depending on your machine, this might take a few minutes.")
smac = SMAC(scenario=scenario, rng=np.random.RandomState(42),
        tae_runner=svm_from_cfg)
incumbent = smac.optimize()
inc_value = svm_from_cfg(incumbent)
print("Optimized Value: %.2f" % (inc_value))
print("Best parameter : ", incumbent.get_dictionary())

Optimizing! Depending on your machine, this might take a few minutes.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


{'C': 1.0, 'kernel': 'poly', 'shrinking': True, 'degree': 3, 'gamma': 'auto'}
0.8751031406834452


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


{'C': 342.8603177140093, 'kernel': 'linear', 'shrinking': False}
0.9440142339532102


...

{'C': 178.1953156683978, 'kernel': 'poly', 'shrinking': False, 'coef0': 9.105183204612164, 'degree': 1, 'gamma': 5.258017005850199}
0.9420137649390415


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


{'C': 333.0243590891621, 'kernel': 'poly', 'shrinking': True, 'coef0': 2.4586429644161365, 'degree': 2, 'gamma': 0.9314552526320006}
0.9436951822198567


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


{'C': 805.9573023422622, 'kernel': 'poly', 'shrinking': False, 'coef0': 6.1318511844018, 'degree': 1, 'gamma': 5.491021967983701}
0.9473073025938182
Optimized Value: 0.05
Best parameter :  {'C': 805.9573023422622, 'kernel': 'poly', 'shrinking': 'false', 'coef0': 6.1318511844018, 'degree': 1, 'gamma': 'value', 'gamma_value': 5.491021967983701}


### Logistic Regression with SMAC on Credit Card Dataset

In [0]:
# Build Configuration Space which defines all parameters and their ranges
cs = ConfigurationSpace()

In [4]:
# Penalty as l1 or l2
penalty = CategoricalHyperparameter("penalty", ["l1", "l2"], default_value="l2")
cs.add_hyperparameter(penalty)

penalty, Type: Categorical, Choices: {l1, l2}, Default: l2

In [5]:
# Common Hyperparameters
C = UniformFloatHyperparameter("C", 0.001, 1000.0, default_value=1.0)
cs.add_hyperparameter(C)

C, Type: UniformFloat, Range: [0.001, 1000.0], Default: 1.0

In [6]:
# penalty specific dual parameter
dual = CategoricalHyperparameter("dual", ["true", "false"], default_value="false")
cs.add_hyperparameter(dual)

# dual is used only when penalty = "l2"
cs.add_condition(InCondition(child=dual, parent=penalty, values=["l2"]))

dual | penalty in {'l2'}

In [0]:
def logreg_from_cfg(cfg):
    print(cfg)
    """ Creates a SVM based on a configuration and evaluates it on the
    iris-dataset using cross-validation.

    Parameters:
    -----------
    cfg: Configuration (ConfigSpace.ConfigurationSpace.Configuration)
        Configuration containing the parameters.
        Configurations are indexable!

    Returns:
    --------
    A crossvalidated mean score for the svm on the loaded data-set.
    """
    # For deactivated parameters, the configuration stores None-values.
    # This is not accepted by the SVM, so we remove them.
    cfg = {k : cfg[k] for k in cfg if cfg[k]}
    # We translate boolean values:
    if(cfg["penalty"]=="l2"):
      cfg["dual"] = True if cfg["dual"] == "true" else False
    
    X, y = creditcard.get_data(target=creditcard.default_target_attribute)
    y[y == "'0'"]=0
    y[y == "'1'"]=1
    
    ICA = decomposition.FastICA(tol=0.2)
    X_new = ICA.fit_transform(X)
    logreg = LogisticRegression(**cfg, random_state=42)
    scores = cross_val_score(logreg, X_new, y, cv=5, scoring="roc_auc")
    print(np.mean(scores))
    return 1-np.mean(scores)  # Minimize!

In [0]:
# Scenario object
scenario = Scenario({"run_obj": "quality",   # we optimize quality (alternatively runtime)
                     "runcount-limit": 10,  # maximum function evaluations
                     "cs": cs,               # configuration space
                     "deterministic": "true"
                     })

In [19]:
# Optimize, using a SMAC-object
print("Optimizing! Depending on your machine, this might take a few minutes.")
smac = SMAC(scenario=scenario, rng=np.random.RandomState(42),
        tae_runner=logreg_from_cfg)
incumbent = smac.optimize()
inc_value = logreg_from_cfg(incumbent)
print("Optimized Value: %.2f" % (inc_value))
print("Best parameter : ", incumbent.get_dictionary())

Optimizing! Depending on your machine, this might take a few minutes.
Configuration:
  C, Value: 1.0
  dual, Value: 'false'
  penalty, Value: 'l2'



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


0.9760163267568283
Configuration:
  C, Value: 372.14745682814066
  penalty, Value: 'l1'



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


0.9677790557900898
Configuration:
  C, Value: 376.15135912829777
  penalty, Value: 'l1'



...

0.9792770816574897
Configuration:
  C, Value: 739.0147045912248
  dual, Value: 'true'
  penalty, Value: 'l2'



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


0.97847293628482
Configuration:
  C, Value: 137.4362714801022
  dual, Value: 'true'
  penalty, Value: 'l2'



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


0.9793791874203188
Optimized Value: 0.02
Best parameter :  {'C': 137.4362714801022, 'penalty': 'l2', 'dual': 'true'}


### TO-DO


1.   Test out for different datasets
2.   Test out different algorithms

